# This notebook constructs a diversified portfolio composed of **SPY, TLT,** and **GLD** using daily closing price data from 2025.
# A portfolio return dataset is built based on market closing prices, and **Value at Risk (VaR)** is estimated using three approaches: **Parametric VaR, Historical VaR, and Monte Carlo simulation.**
# In addition, **Conditional Value at Risk (CVaR)** is calculated under the Monte Carlo framework to evaluate portfolio tail risk.

# All reported VaR and CVaR are **1-day** risk measures based on daily log returns and the specified confidence level. CVaR (Expected Shortfall) is computed as the average portfolio return in the simulated lower tail beyond the VaR cutoff.

In [16]:
pip install yfinance

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
from scipy.stats import norm
import requests
from io import StringIO
import seaborn as sns; sns.set()
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10,6)

# **Data Collection**

# This section retrieves daily closing price data for SPY, TLT, and GLD from Yahoo Finance using the yfinance API.
# The dataset covers the period January 1, 2025 – December 31, 2025.
# Only daily closing prices are retained to construct return series for subsequent portfolio VaR and CVaR analysis.

In [18]:
def get_daily_data(symbol, start_date, end_date):
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(start=start_date, end=end_date)
        if data.empty:
            raise ValueError(f"No data available for {symbol} within the specified date range.")
        return data[['Close']].rename(columns={'Close': 'close'})
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()

symbols = ["SPY", "TLT", "GLD"]
start_date = '2025-01-01'
end_date = '2026-01-01'

stock_data = []

for symbol in symbols:
    stock_data.append(get_daily_data(symbol, start_date, end_date)['close'])

stocks = pd.DataFrame(stock_data).T
stocks.columns = symbols

# Convert index to datetime if it's not already in datetime format
stocks.index = pd.to_datetime(stocks.index)

# Extract just the date part from the index
stocks.index = stocks.index.date

stocks.head()

,SPY,TLT,GLD
2025-01-02,577.854187,83.519257,245.419998
2025-01-03,585.079285,83.252213,243.490005
2025-01-06,588.449707,82.880257,243.190002
2025-01-07,581.797852,81.945587,244.559998
2025-01-08,582.647827,82.050491,245.860001


# **Portfolio Construction**

# Portfolio weights are randomly generated and normalized such that total portfolio exposure equals one:
# $$
\sum_{i=1}^{n} w_i = 1
$$

# The portfolio volatility is computed using the covariance matrix:

# $$
\sigma_p = \sqrt{w^T \Sigma w}
$$

In [19]:
stocks_returns = (np.log(stocks) - np.log(stocks.shift(1))).dropna()
stocks_returns

,SPY,TLT,GLD
2025-01-03,0.012426,-0.003203,-0.007895
2025-01-06,0.005744,-0.004478,-0.001233
2025-01-07,-0.011368,-0.011341,0.005618
2025-01-08,0.001460,0.001279,0.005302
2025-01-10,-0.015385,-0.006647,0.009513
...,...,...,...
2025-12-24,0.003511,0.006039,-0.004143
2025-12-26,-0.000101,-0.003300,0.011609
2025-12-29,-0.003570,0.003754,-0.044504
2025-12-30,-0.001222,-0.002387,0.000727


In [20]:
stocks_returns_mean = stocks_returns.mean()
np.random.seed(42)
weights = np.random.random(len(stocks_returns.columns))
weights /= np.sum(weights)
cov_var = stocks_returns.cov()
port_std = np.sqrt(weights.T.dot(cov_var).dot(weights))

In [21]:
initial_investment = 1e6
conf_level = 0.95

# **Parametric VaR Framework**
# Assuming normally distributed portfolio returns.
# $$
VaR = -V_0 (\mu_p + z_{\alpha}\sigma_p)
$$

In [22]:
def VaR_parametric(initial_investment, conf_level):

    port_mu = weights.dot(stocks_returns_mean)
    z = norm.ppf(1 - conf_level)
    VaR_param = - initial_investment * (port_mu + z * port_std)

    print("Portfolio Parametric VaR is {:.2f}".format(VaR_param))

    return VaR_param


In [23]:
VaR_param = VaR_parametric(initial_investment, conf_level)
VaR_param

Portfolio Parametric VaR is 9541.57


np.float64(9541.568913959503)

# **Historical Simulation VaR**

# Historical VaR is computed using the empirical distribution of portfolio returns without assuming any parametric distribution.
# The portfolio return series is constructed using asset weights, and the VaR is estimated as the lower tail percentile of historical returns.

## $$
VaR = -V_0 \cdot q_{\alpha}
$$

$$
q_{\alpha} = \mathrm{Percentile}_{(1-\text{conf\_level})}(R_p)
$$

$$
R_p = \mathbf{w}^\top \mathbf{r}
$$

In [24]:
def VaR_historical(initial_investment, conf_level):

  port_returns = stocks_returns.dot(weights)
  q = np.percentile(port_returns, (1-conf_level)*100)
  VaR_hist = - initial_investment * q

  print("Portfolio Historical VaR is {:.2f}".format(VaR_hist))
  return VaR_hist

VaR_historical(initial_investment, conf_level)


Portfolio Historical VaR is 8845.11


np.float64(8845.114120741115)

# **Monte Carlo Simulation**

# Monte Carlo VaR is estimated by simulating correlated one-day asset returns using a multivariate normal model calibrated to the historical mean vector and covariance matrix of daily log returns.

# The simulated portfolio return is:
# $$
R_p^{sim} = w^T r^{sim}
$$

# Monte Carlo VaR is computed as:

# $$
VaR = -V_0 \cdot q_\alpha
$$

# where

# $$
q_\alpha = Percentile_{(1-\text{conf\_level})}(R_p^{sim})
$$




In [25]:
# Monte Carlo setup (based on historical returns)
num_reps = 1000  # number of simulated scenarios

# Estimate mean vector and covariance matrix from historical returns
mu = stocks_returns.mean()        # Series, length = number of assets
cov = stocks_returns.cov()        # DataFrame, (n_assets x n_assets)

# Simulate correlated daily returns (num_reps scenarios)
sim_data = pd.DataFrame(
    np.random.multivariate_normal(mu, cov, num_reps),
    columns=stocks_returns.columns
)

sim_data.head()

,SPY,TLT,GLD
0,0.009796,-0.000695,0.013405
1,-0.011084,0.002673,-0.007201
2,-0.008804,0.019181,0.012314
3,0.004103,0.011468,-0.013312
4,-0.006080,0.005039,0.010232


In [26]:
def MC_VaR(initial_investment, conf_level):

    # Step 1: portfolio simulated returns
    port_sim_returns = sim_data.dot(weights)

    # Step 2: percentile cutoff
    q = np.percentile(
        port_sim_returns,
        (1 - conf_level) * 100
    )

    # Step 3: Monte Carlo VaR
    VaR_MC = - initial_investment * q

    print("Monte Carlo Portfolio VaR is {:.2f}".format(VaR_MC))

    return VaR_MC


MC_VaR(initial_investment, conf_level)

Monte Carlo Portfolio VaR is 9357.91


np.float64(9357.907432670556)

# **CVaR Calculation**
# CVaR (Expected Shortfall) measures the average portfolio loss conditional on the portfolio return falling below the VaR threshold.

# Monte Carlo CVaR is computed as:

# $$
CVaR = -V_0 \cdot E(R_p^{sim} \mid R_p^{sim} \le q_\alpha)
$$

# where

# $$
q_\alpha = Percentile_{(1-\text{conf\_level})}(R_p^{sim})
$$

In [27]:

port_sim_returns = sim_data.dot(weights)
q = np.percentile(port_sim_returns, (1 - conf_level) * 100)

tail_losses = port_sim_returns[port_sim_returns <= q]

CVaR_MC = - initial_investment * tail_losses.mean()

print("Monte Carlo Portfolio CVaR is {:.2f}".format(CVaR_MC))

Monte Carlo Portfolio CVaR is 12708.23
